# The Web Search Tool

## Key Concepts
- Built-in Claude tool - fully automated, no implementation required
- Must be enabled in organization settings console first
- Returns multiple block types: text blocks, ServerToolUseBlock (query), WebSearchToolResultBlock (results), WebSearchResultBlock (individual results), citation blocks
- max_uses limits total searches to prevent excessive API calls
- Claude may perform follow-up searches based on initial results
- allowed_domains restricts searches to specific authoritative sources

## Important Code Patterns
- Schema structure: `{"type": "web_search_20250305", "name": "web_search", "max_uses": 5}`
- Domain restriction: `"allowed_domains": ["nih.gov"]` for specific sources
- Include schema in tools array when making API calls
- Response blocks designed for specific UI rendering
- Citations include: source domain, page title, URL, quoted text
- Claude automatically decides when web search needed

## Best Practices
- Set reasonable max_uses (e.g., 5) to control API costs
- Use allowed_domains for reliable/authoritative sources (e.g., nih.gov for medical info)
- Best for: current events, specialized info, fact-checking, research requiring up-to-date data
- Render text blocks as regular content, results as source list, citations inline
- Citations build trust by showing exact sources and quoted text
- Provides transparency about how Claude arrived at answers
- Single search returns multiple results, but Claude may need additional searches

In [ ]:
# Load env variables and create client
from dotenv import load_dotenv
from anthropic import Anthropic

load_dotenv()

client = Anthropic()
model = "claude-sonnet-4-5"

In [ ]:
# Helper functions
from anthropic.types import Message


def add_user_message(messages, message):
    user_message = {
        "role": "user",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(user_message)


def add_assistant_message(messages, message):
    assistant_message = {
        "role": "assistant",
        "content": message.content if isinstance(message, Message) else message,
    }
    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
        "stop_sequences": stop_sequences,
    }

    if tools:
        params["tools"] = tools

    if system:
        params["system"] = system

    message = client.messages.create(**params)
    return message


def text_from_message(message):
    return "\n".join([block.text for block in message.content if block.type == "text"])

In [ ]:
web_search_schema = {
    "type": "web_search_20250305",
    "name": "web_search",
    "max_uses": 5,
    "allowed_domains": ["nih.gov"],
}

In [ ]:
messages = []
add_user_message(
    messages,
    """
    What's the best exercise for gaining leg muscle?
    """,
)
response = chat(messages, tools=[web_search_schema])
response